In [1]:
%load_ext autoreload
%autoreload 2
import os
import gc
import sys
import glob
import datetime
import pandas as pd
import numpy as np
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
sys.path.append(f"../py/")
import MS_utils
import utils, ml_utils, kaggle_utils
from utils import logger_func
from sklearn.model_selection import KFold
from lofo import LOFOImportance, plot_importance
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Columns
key, target, ignore_list = MS_utils.get_basic_var()

# Basic Args
seed = 1208
fold_n = 3
lofo_list = []
result_list = []
ex_list = []

In [50]:
import shutil
import re
pd.set_option('max_rows', 100)

date = '0311'
sampling = 600000
sampling = 300000
base = utils.read_df_pkl('../input/base_Av*')

#========================================================================
# featureの種類別に検証したい時、前方一致でグループを作り、順にループする 
# feat_path = '../features/4_winner/*.gz'
# feat_path = '../features/1_first_valid/f*.gz'
# feat_path_list = glob.glob(feat_path)
# fname_list = []
# for path in feat_path_list:
#     filename = re.search(r'/([^/.]*).gz', path).group(1)[:25]
#     if filename in ex_list:
#         shutil.move(path, '../features/2_second_valid/')
#         continue
#     fname_list.append(filename)
# fname_list = list(set(fname_list))
#========================================================================
fname_list = ['f']

for fname in fname_list:
    feat_path = f'../features/1_first_valid/{fname}*.gz'

    #========================================================================
    # Train Test Load
    try:
        train, test = MS_utils.get_dataset(base=base, feat_path=feat_path)
    except ValueError:
        print(f"Not Exist: {fname}")
        continue
    vi_col = 'f000_AvSigVersion'
    ignore_list = [key, target, 'country_group', vi_col]
    
    train.sort_values(vi_col, inplace=True)
    test.sort_values(vi_col, inplace=True)
    train.drop(vi_col, axis=1, inplace=True)
    test.drop(vi_col, axis=1, inplace=True)
    
    # extract a sample of the data
    group_list = sorted(train['country_group'].unique().tolist())
    print(group_list)
    #========================================================================
    
    features = [col for col in train.columns if col.count('f0')]
    base_mean = pd.DataFrame([])
    for group in group_list:
        start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
    
        if sampling:
            sample_train = train[train['country_group']==group].sample(n=sampling)
        else:
            sample_train = train[train['country_group']==group]
        print(f"Group {group} | Train Shape: {sample_train.shape}")
    
        # define the validation scheme
        cv = KFold(n_splits=fold_n, shuffle=False, random_state=seed)
        use_cols = [col for col in sample_train.columns if col not in ignore_list] + [target]
    
        # define the binary target and the features
    
        # define the validation scheme and scorer. The default model is LightGBM
        lofo_imp = LOFOImportance(sample_train[use_cols], features, target, cv=cv, scoring="roc_auc", n_jobs=-1)
    
        # get the mean and standard deviation of the importances in pandas format
        importance_df = lofo_imp.get_importance()
        # plot the means and standard deviations of the importances
    #     fig = plt.figure(figsize=(10, 20))
    #     fig.patch.set_facecolor('white')
#         sns.set_style("whitegrid")
#         plot_importance(importance_df, figsize=(20, 20))
#         plt.savefig(f'../lofo_png/{start_time[4:12]}_lofo_row{len(sample_train)}_feat{len(sample_train.columns)}_gr{group}.png')
#         plt.show()
        
        importance_df.columns = [f"gr{group}_{col}" if col.count('imp') else col  for col in importance_df.columns]
        lofo_list.append(importance_df)
        
        #========================================================================
        # 現状のグループ間比較 
        df = importance_df.copy()
        if 'feature' in df.columns:
            df.set_index('feature', inplace=True)
        if len(base_mean)==0:
            base_mean = df.copy()
        else:
            base_mean = base_mean.join(df)
                
        mean_cols = [col for col in base_mean.columns if col.count('_mean')]
        base_mean = base_mean[mean_cols]
        for col in mean_cols:
            base_mean[col] = base_mean[col].map(lambda x: np.round(x, 4))
        base_mean['cnt_minus'] = (base_mean[mean_cols]<0).sum(axis=1)
        display(base_mean)
        del df
        gc.collect()
        #========================================================================
        
    result_list.append(base_mean)
    result = pd.concat(result_list, axis=0)
    result.to_csv(f"../output/{date}_TMP_lofo_group_fold4_validfeature.csv")
    print(len(lofo_list))
    feat_path_list = glob.glob(feat_path)
    for path in feat_path_list:
        shutil.move(path, '../features/2_second_valid/')

100%|██████████| 3/3 [00:01<00:00,  2.70it/s]


(16774736, 5)
(8921483, 74) (7853253, 74)
Categorical: ['f000_EngineVersion', 'f015_f000_AVProductStatesIdentifier-f000_Census_OSBuildRevision', 'f015_f000_Census_PrimaryDiskTotalCapacity-f000_RtpStateBitfield', 'f015_f000_Processor-f000_RtpStateBitfield', 'f015_f000_Census_OSArchitecture-f000_RtpStateBitfield', 'f015_f000_AVProductStatesIdentifier-f000_Census_ActivationChannel', 'f015_f000_Census_OSBuildRevision-f000_SmartScreen', 'f015_f000_Census_OSBuildRevision-f000_IeVerIdentifier']
(8921483, 74) (7853253, 74)
[1, 2, 3, 4, 5, 6, 7, 8, 9]
Group 1 | Train Shape: (300000, 73)


A Jupyter Widget

,gr1_importance_mean,cnt_minus
feature,,
f014_f000_Census_OSArchitecture-f000_Census_TotalPhysicalRAM_count,0.0009,0
f014_f000_Census_ActivationChannel-f000_CountryIdentifier_count,0.0009,0
f014_f000_Census_OSUILocaleIdentifier-f000_CountryIdentifier_count,0.0007,0
f014_f000_CountryIdentifier-f000_IsProtected_count,0.0005,0
f015_f000_Census_PrimaryDiskTotalCapacity-f000_RtpStateBitfield,0.0004,0
f014_f000_Census_OSEdition-f000_Wdft_IsGamer_count,0.0004,0
f012_AppVersion-3,0.0003,0
f014_f000_AvSigVersion-f000_Census_OSBuildRevision_count,0.0002,0
f014_f000_Census_OSVersion-f000_OsBuildLab_count,0.0002,0


Group 2 | Train Shape: (300000, 73)


...

,gr1_importance_mean,gr2_importance_mean,gr3_importance_mean,gr4_importance_mean,gr5_importance_mean,gr6_importance_mean,gr7_importance_mean,gr8_importance_mean,cnt_minus
feature,,,,,,,,,
f014_f000_Census_OSArchitecture-f000_Census_TotalPhysicalRAM_count,0.0009,0.0010,0.0006,0.0008,0.0012,0.0008,0.0014,0.0012,0
f014_f000_Census_ActivationChannel-f000_CountryIdentifier_count,0.0009,0.0008,0.0005,-0.0000,0.0003,0.0003,0.0002,0.0003,0
f014_f000_Census_OSUILocaleIdentifier-f000_CountryIdentifier_count,0.0007,0.0009,0.0010,0.0002,0.0008,0.0001,0.0007,0.0004,0
f014_f000_CountryIdentifier-f000_IsProtected_count,0.0005,0.0000,0.0000,0.0002,0.0001,0.0001,0.0002,0.0004,0
f015_f000_Census_PrimaryDiskTotalCapacity-f000_RtpStateBitfield,0.0004,0.0002,0.0004,0.0004,0.0004,0.0004,0.0003,0.0003,0
f014_f000_Census_OSEdition-f000_Wdft_IsGamer_count,0.0004,0.0002,0.0006,0.0001,0.0003,0.0001,0.0005,0.0004,0
f012_AppVersion-3,0.0003,-0.0001,0.0002,-0.0001,-0.0001,-0.0002,0.0000,0.0001,4
f014_f000_AvSigVersion-f000_Census_OSBuildRevision_count,0.0002,-0.0001,0.0003,0.0002,0.0002,0.0004,0.0001,0.0006,1
f014_f000_Census_OSVersion-f000_OsBuildLab_count,0.0002,0.0002,0.0003,-0.0000,0.0001,0.0005,0.0003,-0.0001,1


Group 9 | Train Shape: (300000, 73)


A Jupyter Widget

,gr1_importance_mean,gr2_importance_mean,gr3_importance_mean,gr4_importance_mean,gr5_importance_mean,gr6_importance_mean,gr7_importance_mean,gr8_importance_mean,gr9_importance_mean,cnt_minus
feature,,,,,,,,,,
f014_f000_Census_OSArchitecture-f000_Census_TotalPhysicalRAM_count,0.0009,0.0010,0.0006,0.0008,0.0012,0.0008,0.0014,0.0012,0.0008,0
f014_f000_Census_ActivationChannel-f000_CountryIdentifier_count,0.0009,0.0008,0.0005,-0.0000,0.0003,0.0003,0.0002,0.0003,0.0005,0
f014_f000_Census_OSUILocaleIdentifier-f000_CountryIdentifier_count,0.0007,0.0009,0.0010,0.0002,0.0008,0.0001,0.0007,0.0004,0.0011,0
f014_f000_CountryIdentifier-f000_IsProtected_count,0.0005,0.0000,0.0000,0.0002,0.0001,0.0001,0.0002,0.0004,-0.0000,0
f015_f000_Census_PrimaryDiskTotalCapacity-f000_RtpStateBitfield,0.0004,0.0002,0.0004,0.0004,0.0004,0.0004,0.0003,0.0003,0.0001,0
f014_f000_Census_OSEdition-f000_Wdft_IsGamer_count,0.0004,0.0002,0.0006,0.0001,0.0003,0.0001,0.0005,0.0004,0.0009,0
f012_AppVersion-3,0.0003,-0.0001,0.0002,-0.0001,-0.0001,-0.0002,0.0000,0.0001,-0.0003,5
f014_f000_AvSigVersion-f000_Census_OSBuildRevision_count,0.0002,-0.0001,0.0003,0.0002,0.0002,0.0004,0.0001,0.0006,0.0002,1
f014_f000_Census_OSVersion-f000_OsBuildLab_count,0.0002,0.0002,0.0003,-0.0000,0.0001,0.0005,0.0003,-0.0001,0.0000,1


18


In [55]:
print('Complete!')
result[result['cnt_minus']>4]

Complete!


,gr1_importance_mean,gr2_importance_mean,gr3_importance_mean,gr4_importance_mean,gr5_importance_mean,gr6_importance_mean,gr7_importance_mean,gr8_importance_mean,gr9_importance_mean,cnt_minus
feature,,,,,,,,,,
f014_f000_Census_OEMModelIdentifier-f000_Census_SystemVolumeTotalCapacity_count,-0.0001,0.0009,-0.0003,0.0003,-0.0001,-0.0001,0.0004,-0.0003,-0.0001,6
f015_f000_Census_OSInstallTypeName-f000_Census_PrimaryDiskTotalCapacity,-0.0001,0.0003,-0.0002,0.0002,-0.0001,-0.0001,0.0002,0.0001,-0.0003,5
f014_f000_Census_OSBuildRevision-f000_Census_OSInstallTypeName_count,-0.0001,0.0003,-0.0001,0.0002,-0.0002,0.0002,0.0004,-0.0003,-0.0001,5
f014_f000_AvSigVersion-f000_Wdft_RegionIdentifier_count,-0.0002,0.0001,-0.0002,0.0002,0.0002,-0.0005,0.0002,-0.0002,-0.0002,5
f014_f000_Census_ActivationChannel-f000_EngineVersion_count,-0.0002,0.0005,-0.0000,0.0004,-0.0003,-0.0003,0.0004,-0.0003,-0.0001,5
f014_f000_EngineVersion-f000_SmartScreen_count,-0.0002,0.0002,-0.0002,-0.0000,-0.0002,-0.0003,0.0006,-0.0002,-0.0002,6
f014_f000_OsBuildLab-f000_Wdft_RegionIdentifier_count,-0.0002,0.0004,-0.0002,0.0002,-0.0001,-0.0000,-0.0000,-0.0002,-0.0002,5
f014_f000_AVProductStatesIdentifier-f000_Census_OEMModelIdentifier_count,-0.0002,0.0004,0.0001,-0.0002,-0.0002,-0.0001,0.0003,-0.0002,0.0001,5
f014_f000_OsBuildLab-f000_RtpStateBitfield_count,-0.0002,0.0006,-0.0002,0.0000,-0.0002,-0.0002,0.0001,-0.0002,0.0003,5


In [53]:
result = pd.concat(result_list, axis=0)
for feat in result.index.tolist():
    ex_list.append(feat[:25])
ex_list = list(set(ex_list))
result.to_csv(f"../output/{date}_lofo_group_fold4_feat130-Good-feats_valid-feature{sample_train.shape[1]}_sample{sample_train.shape[0]}.csv")

In [47]:
tmp = result[result['cnt_minus']<4]
gr_cols = [col for col in result.columns if col[:2]=='gr' and not(col.count('gr1'))]
feat_list = []

tmp_1 = tmp.loc[(tmp<-0.0004).sum(axis=1)>0, :][gr_cols].sum(axis=1)
feat_list += tmp_1.loc[tmp_1>0.0002].index.tolist()

tmp_2 = tmp[tmp['cnt_minus']==3][gr_cols].sum(axis=1)
feat_list += tmp_2[tmp_2>0.0009].index.tolist()

tmp_3 = tmp[tmp['cnt_minus']<=2][gr_cols].sum(axis=1)
feat_list += tmp_3[tmp_3>0.0009].index.tolist()

move_fname_list = list(set(feat_list))

In [49]:
# result = pd.read_csv('../output/0307_MS_lofo_TE_valid_490feature.csv').set_index('feature')
import re
import shutil
#========================================================================
# 検証したfeatureの移動する 
#========================================================================
# gr_imp_cols = [col for col in result.columns if col[:2]=='gr']
# move_fname_list =  result[(result['cnt_minus']<4) & (result[gr_imp_cols].max(axis=1)>0.0004)].index.tolist()

# path_list = glob.glob('../features/4_winner/*.gz')
path_list = glob.glob('../features/6_subset/*.gz')

for path in path_list:
    filename = re.search(r'/([^/.]*).gz', path).group(1)
    for col in move_fname_list:
#         if path.count(col) and col[:4]==filename[:4]:
        if col==filename:
#             shutil.move(path, '../features/2_second_valid/')
#             shutil.move(path, '../features/1_first_valid/')
            shutil.move(path, '../features/3_third_valid/')